In [1]:
import subprocess
import sys
import json
from pathlib import Path

# Create a temporary Python script
script_content = '''
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig, CacheMode
import asyncio
import json
import sys

urls_to_crawl = [
    "https://docs.llamaindex.ai/en/stable/understanding/",
]

config = CrawlerRunConfig(
    cache_mode=CacheMode.BYPASS,
    page_timeout=80000,
    word_count_threshold=50,
)

async def crawl_website():
    data_res = {"data": []}

    async with AsyncWebCrawler(headless=True, verbose=False) as crawler:  # Set verbose=False
        results = await crawler.arun_many(urls_to_crawl, config=config)

        for result in results:
            if result.success:
                title = result.metadata.get("title", "")
                if not title and result.markdown:
                    lines = result.markdown.raw_markdown.split("\\n")
                    for line in lines:
                        if line.startswith("#"):
                            title = line.strip("#").strip()
                            break

                data_res["data"].append(
                    {
                        "text": result.markdown.raw_markdown if result.markdown else "",
                        "meta": {"url": result.url, "meta": {"title": title}},
                    }
                )

    return data_res

if __name__ == "__main__":
    data_res = asyncio.run(crawl_website())
    # Write to stderr for debugging, stdout for data
    print(json.dumps(data_res), file=sys.stdout)
'''

# Write script to temp file
script_path = Path("temp_crawler.py")
script_path.write_text(script_content)

# Run as subprocess
try:
    result = subprocess.run(
        [sys.executable, str(script_path)],
        capture_output=True,
        text=True,
        timeout=120
    )

    if result.returncode == 0:
        # Parse the JSON output
        data_res = json.loads(result.stdout.strip())
        print("Successfully crawled!")
        print(f"Number of results: {len(data_res['data'])}")
        if data_res['data']:
            print(f"First result URL: {data_res['data'][0]['meta']['url']}")
            print(f"Text length: {len(data_res['data'][0]['text'])} characters")
    else:
        print(f"Error: {result.stderr}")
finally:
    # Clean up
    if script_path.exists():
        script_path.unlink()

JSONDecodeError: Expecting value: line 1 column 2 (char 1)